In [1]:
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras import utils as np_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# upload txt files for our data
# All Rock
rock1 = open('/content/drive/My Drive/Colab Notebooks/AllRock.txt', 'r').read()
rock = ''.join([i for i in rock1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Pop
pop1 = open('/content/drive/My Drive/Colab Notebooks/AllPop.txt', 'r').read()
pop = ''.join([i for i in pop1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Country
country1 = open('/content/drive/My Drive/Colab Notebooks/AllCountry.txt', 'r').read()
country = ''.join([i for i in country1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')
# All Rap
rap1 = open('/content/drive/My Drive/Colab Notebooks/AllLyrics.txt', 'r').read()
rap = ''.join([i for i in rap1 if not i.isdigit()]).replace("\n", " ").lower().replace(".", "").replace("!", "").replace("?", "").replace(",", "").replace("", "").replace("'", "").replace(")", "").replace("(", "").split(' ')

In [16]:
# create samples of 10 words each for each genre - this is our estimate length of line
SONG_LENGTH = 10
# Rock
Rock = [rock[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rock)/int(SONG_LENGTH)))]
# Country
Country = [country[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(country)/int(SONG_LENGTH)))]
# Pop
Pop = [pop[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(pop)/int(SONG_LENGTH)))]
# Rap
Rap = [rap[i*SONG_LENGTH:(i+1)*SONG_LENGTH] for i in range(0,int(len(rap)/int(SONG_LENGTH)))]

In [17]:
# joining the strings in the samples
ds_rock = [' '.join(Rock[i]) for i in range(len(Rock))]
ds_country = [' '.join(Country[i]) for i in range(len(Country))]
ds_pop = [' '.join(Pop[i]) for i in range(len(Pop))]
ds_rap = [' '.join(Rap[i]) for i in range(len(Rap))]

In [18]:
# make a list here where each sample has it's genre number
# rock 0, country 1, pop 2, rap 3
ds_ro = []
genre = 0
for sample in ds_rock:
  ds_ro.append([genre, sample])

ds_co = []
genre = 1
for sample in ds_country:
  ds_co.append([genre, sample])

ds_po = []
genre = 2
for sample in ds_pop:
  ds_po.append([genre, sample])

ds_ra = []
genre = 3
for sample in ds_rap:
  ds_ra.append([genre, sample])

ds = ds_ro+ds_co+ds_po+ds_ra

ds = np.array(ds)
print('Genres: ', ds[:, 0])
print('Lyrics: ', ds[:, 1])

Genres:  ['0' '0' '0' ... '3' '3' '3']
Lyrics:  ['yesterday all my troubles seemed so far away now it'
 'looks as though theyre here to stay oh i believe'
 'in yesterday suddenly im not half the man i used' ...
 'kick me when im down but im up again scorchin'
 'hot forcin my way up in the door to kill'
 'the bullpoop like a matador keep your hands high what']


In [19]:
x = ds[:, 1]
y = ds[:, 0]

In [20]:
 # tokenize here
tk = Tokenizer(num_words= 1000, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(x)
x = tk.texts_to_sequences(x)
x = sequence.pad_sequences(x, maxlen=200)

In [21]:
# classification category
labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)

In [22]:
# one hot encoding 
y = np_utils.to_categorical(y, num_classes= 4)

In [23]:
np.random.seed(200)
indices = np.arange(len(x))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [24]:
index_from=3
start_char = 1
if start_char is not None:
  x = [[start_char] + [w + index_from for w in x1] for x1 in x]
  elif index_from:
x = [[w + index_from for w in x1] for x1 in x]

SyntaxError: ignored

In [25]:
num_words = None
if not num_words:
  num_words = max([max(x1) for x1 in x])

  oov_char = 2
  skip_top = 0

  if oov_char is not None:
    x = [[w if (skip_top <= w < num_words) else oov_char for w in x1] for x1 in x]
  else:
    x = [[w for w in x1 if (skip_top <= w < num_words)] for x1 in x]
        
# split data here
test_split = 0.2
idx = int(len(x) * (1 - test_split))
x_train, y_train = np.array(x[:idx]), np.array(y[:idx])
x_test, y_test = np.array(x[idx:]), np.array(y[idx:])

In [26]:
x_train = sequence.pad_sequences(x_train, maxlen=201)
x_test = sequence.pad_sequences(x_test, maxlen=201)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (42039, 201)
x_test shape: (10510, 201)


In [28]:
max_features = 1000
maxlen = 201
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250


# CNN here
print('Building model...')
model = Sequential()

model.add(Embedding(max_features,embedding_dims,input_length=maxlen))
model.add(Dropout(0.2))

# add Convolution1D
model.add(Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1))
# max pooling:
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=32,epochs=5,validation_data=(x_test, y_test))

Building model...
Epoch 1/5
1314/1314 [==============================] - 53s 40ms/step - loss: 0.9923 - accuracy: 0.6147 - val_loss: 0.8607 - val_accuracy: 0.6598
Epoch 2/5
1314/1314 [==============================] - 52s 40ms/step - loss: 0.8227 - accuracy: 0.6726 - val_loss: 0.8118 - val_accuracy: 0.6825
Epoch 3/5
1314/1314 [==============================] - 52s 40ms/step - loss: 0.7569 - accuracy: 0.7074 - val_loss: 0.7790 - val_accuracy: 0.6982
Epoch 4/5
1314/1314 [==============================] - 52s 40ms/step - loss: 0.7135 - accuracy: 0.7240 - val_loss: 0.7468 - val_accuracy: 0.7127
Epoch 5/5
1314/1314 [==============================] - 52s 40ms/step - loss: 0.6608 - accuracy: 0.7501 - val_loss: 0.7413 - val_accuracy: 0.7157


In [29]:
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

329/329 [==============================] - 3s 9ms/step - loss: 0.7413 - accuracy: 0.7157
Test set
  Loss: 0.741
  Accuracy: 0.716


In [30]:
def classify_string(input):
  lyric = [str(input)]
  seq = tk.texts_to_sequences(lyric)
  pred = model.predict(seq)
  labels = ['rock', 'country', 'pop', 'rap']
  print(labels[np.argmax(pred)])

In [31]:
def classify_list(input):
  for lyric in input:
    classify_string(lyric)

In [32]:
# markov generated lyrics
lyrics = ['Bumpin i meant for you call my ninja like',
 'Biz dont take their baby mommas ninja frick you nasty boy you',
 'Shifty sticks and pray and flee the frick all of you',
 'Glocks but all ill die slow',
 'Wondering if im askin blunt sip champagne range rover been outside for',
 'And youre so take that crown two pounds you know',
 'Publishing i thought i get witcha can i could cop',
 'Miss the more cause you in the right one',
 'Onyx and them hoes i love',
 'Gat call me puff daddy biggie gots ta like',
 'Everything around me shit b***** in ya imma stay yappin when',
 'Hum all about fingers in the loot im',
 'Rollem up heard whos this yeah keep on top sky is',
 'Drunk of ninjaz from now drop to',
 'Declinin windin like flypaper neighbor slow down',
 'Expensive cars i tote my crew i only got enough heart',
 'Lame dudes whos next move but the drugs to spit phrases thatll',
 'Guy well its cool and your poop so hard to',
 'Clap wit my life in ma little nasty boy',
 'Dial you should too much better man played',
 'Lali like that you frick doin all mcs have']

classify_list(lyrics)

rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap
rap


In [33]:
# lstm generated lyrics
lyrics2 = ['in the veins hard to explain how i maintain', 
  'to put my back in the house so i can i wanna flaunt you thats right', 
  'with the grime of my ninja frick',
  'with the ds crept in blastin him you dont want to slit the clits alot',
  'used to lick the clits a lot of problems never be the beamer with the goldie sound',
  'like a steelo not my steelo oh no thats not my my steelo oh i steelo not my steelo oh no',
  'thats not my no steelo bust my no dough day but this sittin bodies not my']

classify_list(lyrics2)

rap
rap
rap
rap
rap
pop
rap
